# Project: Wrangling and Analyze Data

In [1]:
# Import necessary python libraries.
import pandas as pd
import requests 
import os

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
twitter_archive_enhanced = pd.read_csv('data/twitter-archive-enhanced.csv')

In [3]:
twitter_archive_enhanced.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [4]:
# Save file to download folder
folder_name = 'data'

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(folder_name, url.split('/')[-1]), 'wb') as file:
    file.write(response.content)

ConnectionError: HTTPSConnectionPool(host='d17h27t6h515a5.cloudfront.net', port=443): Max retries exceeded with url: /topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025035EE0B50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
image_prediction = pd.read_csv('data/image-predictions.tsv', sep='\t')
image_prediction.head()

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [5]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = twitter_archive_enhanced.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

1: 892420643555336193


AttributeError: module 'tweepy' has no attribute 'TweepError'

In [56]:
# List of dictionaries to build file by file and later convert to a DataFrame
folder_name = 'data'
all_tweet = []
df_list = []

for line in open(os.path.join(folder_name, 'tweet-json.txt'), encoding='utf-8'):
    all_tweet.append(json.loads(line))
    print(all_tweet[0]['source'])
    w_source=all_tweet[0]['source']
    source=w_source[w_source.find('rel="nofollow">') + 15:-4]
    #source=w_source[w_source.find('rel="nofollow">') + 15:-4]
    print(source)
    break
    for tweet in all_tweet:
        tweet_id = tweet['id']
        whole_tweet = tweet['full_text']
        only_url = whole_tweet[whole_tweet.find('https'):] 
        retweet_count = tweet['retweet_count']
        favorite_count = tweet['favorite_count']
        followers_count = tweet['user']['followers_count']
        friends_count = tweet['user']['friends_count']
        whole_source = tweet['source']
        
        
        df_list.append({'tweet_id': tweet_id,
                        'only_url': only_url,
                        'retweet_count': retweet_count,
                        'favorite_count': favorite_count,
                        'followers_count': followers_count,
                        'friends_count': friends_count,
                        'whole_source': whole_source})
        
tweet_json = pd.DataFrame(df_list, columns = ['tweet_id', 'only_url', 'retweet_count', 'favorite_count', 
                                     'followers_count', 'friends_counts', 'whole_source'])

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
Twitter for iPhone


In [39]:
tweet_json

,tweet_id,only_url,retweet_count,favorite_count,followers_count,friends_counts,whole_source
0,892420643555336193,https://t.co/MgUWQ76dJU,8853,39467,3200889,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
1,892420643555336193,https://t.co/MgUWQ76dJU,8853,39467,3200889,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
2,892177421306343426,https://t.co/0Xxu71qeIV,6514,33819,3200889,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
3,892420643555336193,https://t.co/MgUWQ76dJU,8853,39467,3200889,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
4,892177421306343426,https://t.co/0Xxu71qeIV,6514,33819,3200889,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
...,...,...,...,...,...,...,...
2771830,666049248165822465,https://t.co/4B7cOc1EDq,41,111,3201018,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
2771831,666044226329800704,https://t.co/DWnyCjf2mx,147,311,3201018,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
2771832,666033412701032449,https://t.co/y671yMhoiR,47,128,3201018,NaN,"<a href=""http://twitter.com/download/iphone"" r..."
2771833,666029285002620928,https://t.co/r7mOb2m0UI,48,132,3201018,NaN,"<a href=""http://twitter.com/download/iphone"" r..."


## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization